Requirements

1. Input the Excel workbook containing the four worksheets of data
2. Union the Flow Card and Non-Flow card data sets together
3. Create a data field to show whether the seat was booked by someone with the Flow Card or not: Call this field 'Flow Card?'
4. Aggregate the Seat Bookings to count how many bookings there are for: Each Seat, In each Row, In each Class, For Flow and Non-Flow Card holders
5. Join on the Seating Plan data to ensure you have a data set for every seat on the plane, even if it hasn't been book: Only return the records for the seats that haven't been booked
6. Output the data set showing what seat, rows and class have NOT been booked

In [2]:
import pandas as pd

1. Input the Excel workbook containing the four worksheets.
2. Union the Flow card & non flow card data sets together.

In [4]:
sheets = ['Flow Card','Non_flow Card','Non_flow Card2']

alloc_df = pd.concat([ pd.read_excel('Preppin Data Inputs/PD 2024 Wk 4 Input.xlsx', sheet_name=s).assign(sheet_name=s) for s in sheets ], ignore_index=True)

In [5]:
alloc_df

,CustomerID,Seat,Row,Class,sheet_name
0,654,2,2,FC,Flow Card
1,466,4,5,FC,Flow Card
2,27,4,3,FC,Flow Card
3,519,1,4,FC,Flow Card
4,933,2,3,FC,Flow Card
...,...,...,...,...,...
29208,3005,7,35,E,Non_flow Card2
29209,4685,4,27,E,Non_flow Card2
29210,2512,8,38,E,Non_flow Card2
29211,3863,4,37,E,Non_flow Card2


In [6]:
plan_df = pd.read_excel('Preppin Data Inputs/PD 2024 Wk 4 Input.xlsx', sheet_name='Seat Plan')

In [7]:
plan_df

,Class,Seat,Row
0,FC,1,1
1,FC,2,1
2,FC,3,1
3,FC,4,1
4,FC,1,2
...,...,...,...
291,E,6,42
292,E,7,42
293,E,8,42
294,E,9,42


3. Create a data field to show whether the seat was booked by someone with the Flow Card or not: Call this field 'Flow Card?'

In [9]:
alloc_df['Flow Card?'] = alloc_df['sheet_name'].apply(lambda x: 'No' if 'Non' in x else 'Yes')

In [10]:
alloc_df

,CustomerID,Seat,Row,Class,sheet_name,Flow Card?
0,654,2,2,FC,Flow Card,Yes
1,466,4,5,FC,Flow Card,Yes
2,27,4,3,FC,Flow Card,Yes
3,519,1,4,FC,Flow Card,Yes
4,933,2,3,FC,Flow Card,Yes
...,...,...,...,...,...,...
29208,3005,7,35,E,Non_flow Card2,No
29209,4685,4,27,E,Non_flow Card2,No
29210,2512,8,38,E,Non_flow Card2,No
29211,3863,4,37,E,Non_flow Card2,No


4. Aggregate the Seat Bookings to count how many bookings there are for: Each Seat, In each Row, In each Class, For Flow and Non-Flow Card holders

In [12]:
agg_df = alloc_df.groupby(['Seat','Row','Class','Flow Card?'])['CustomerID'].count()

In [13]:
agg_df = agg_df.reset_index()

In [14]:
agg_df

,Seat,Row,Class,Flow Card?,CustomerID
0,1,1,FC,No,38
1,1,1,FC,Yes,20
2,1,2,FC,No,39
3,1,2,FC,Yes,20
4,1,3,FC,No,38
...,...,...,...,...,...
573,10,40,E,Yes,37
574,10,41,E,No,68
575,10,41,E,Yes,32
576,10,42,E,No,90


5. Join on the Seating Plan data to ensure you have a data set for every seat on the plane, even if it hasn't been book: Only return the records for the seats that haven't been booked

In [16]:
final_df = agg_df.merge(right=plan_df, how='right', on=['Class','Seat','Row'])

In [33]:
final_df = final_df[final_df['CustomerID'].isnull()]

In [35]:
final_df

,Seat,Row,Class,Flow Card?,CustomerID
300,5,28,E,NaN,NaN
381,6,32,E,NaN,NaN
458,5,36,E,NaN,NaN
479,6,37,E,NaN,NaN
536,5,40,E,NaN,NaN
537,6,40,E,NaN,NaN
554,5,41,E,NaN,NaN


6. Output the data set showing what seat, rows and class have NOT been booked

In [19]:
final_df.to_csv('Preppin Data Outputs/pd2024wk4_output.csv', index=False)